This notebook demonstrates preparing a ligand-protein system for OpenMM using the OpenFF Sage forcefield for the ligand and Amber99SB for the protein. 

This is a stripped down version of the "tookit_showcase" from OpenFF Toolkit.

In [1]:
import openmm.app
import openmm
from openmmforcefields.generators import SMIRNOFFTemplateGenerator

from openff.toolkit.topology import Molecule
from openff.units.openmm import ensure_quantity

In [2]:
# load both the Protein and Ligand structures
# OpenMM is used to correctly interpret the protein structure
# OpenFF is used for the ligand to prepare for the upcoming forcefield parametrisation

protein = openmm.app.PDBFile('molecules/3l9h_prepared.pdb')
ligand = Molecule('molecules/chembl_1078774.sdf')

In [3]:
# Load protein and water force field parameters
omm_forcefield = openmm.app.ForceField("amber99sb.xml", "tip3p.xml")
# The TemplateGenerator caters for the missing parameters caused by the ligand molecule
smirnoff = SMIRNOFFTemplateGenerator(forcefield="openff-2.0.0.offxml", molecules=[ligand])
omm_forcefield.registerTemplateGenerator(smirnoff.generator)

In [4]:
ligand.conformers

[array([[ 12.81459999,   0.80800003,   0.0689    ],
        [ 11.91940022,  -0.14740001,  -0.75419998],
        [ 10.46850014,   0.366     ,  -0.77810001],
        [ 12.47529984,  -0.2208    ,  -2.17810011],
        [ 11.88860035,  -1.579     ,  -0.1851    ],
        [ 10.85159969,  -2.45409989,  -0.54689997],
        [ 10.79469967,  -3.74349999,  -0.055     ],
        [ 11.81169987,  -4.20359993,   0.76440001],
        [ 12.88070011,  -3.36919999,   1.1329    ],
        [ 12.88199997,  -2.0552001 ,   0.67629999],
        [ 14.02859974,  -3.87409997,   1.98640001],
        [ 13.80760002,  -3.44849992,   3.33260012],
        [ 14.87580013,  -3.7894001 ,   4.21649981],
        [ 15.07149982,  -5.31809998,   4.25670004],
        [ 15.23139954,  -5.90089989,   2.84360003],
        [ 14.11810017,  -5.40789986,   1.8901    ],
        [ 12.71850014,  -6.04879999,   2.12649989],
        [ 12.76119995,  -7.5479002 ,   2.03509998],
        [ 13.24530029,  -8.18929958,   0.8854    ],
        [ 13

In [5]:
# Combine structures and solvate
modeller = openmm.app.Modeller(protein.topology, protein.positions)
modeller.add(ligand.to_topology().to_openmm(),
             ensure_quantity(ligand.conformers[0], 'openmm'))

# solvate it in 0.15 M NaCl solution
modeller.addSolvent(
    omm_forcefield,
    model="tip3p",
    padding=4.0 * openmm.unit.angstrom,  # 4.0 not enough, but works quickly
    ionicStrength=0.15 * openmm.unit.molar,
)

In [6]:
system = omm_forcefield.createSystem(
    modeller.getTopology(),
    nonbondedMethod=openmm.app.PME,
    nonbondedCutoff=9 * openmm.unit.angstrom,
    constraints=openmm.app.HBonds,
    rigidWater=True,
)

In [7]:
# Create a Simulation from our newly prepared System
# Construct and configure a Langevin integrator at 300 K with an appropriate friction constant and time-step
integrator = openmm.LangevinIntegrator(
    300 * openmm.unit.kelvin, 1 / openmm.unit.picosecond, 0.002 * openmm.unit.picoseconds
)

# Combine the topology, system, integrator and initial positions into a simulation
simulation = openmm.app.Simulation(modeller.getTopology(), system, integrator)
simulation.context.setPositions(modeller.getPositions())